In order to run slicer with jupyter notebook firt pip_install('pip==new_version')
then pip_install('ipywidgets')

In [1]:
import sys
import os
from os.path import split, join
base_dir = split(os.getcwd())[0]
if base_dir not in sys.path:
    sys.path.append(base_dir)
import pickle
import slicer
import JupyterNotebooksLib as slicernb
import json
import pandas as pd

from slicer.ScriptedLoadableModule import *
from DICOMLib import DICOMUtils

slicernb.AppWindow.setWindowSize(scale=0.5)
slicernb.showSliceViewAnnotations(False)
import matplotlib.pyplot as plt
#matplotlib.use('WXAgg')

#  Functions

In [2]:
import os
import subprocess
FILEBROWSER_PATH = os.path.join(os.getenv('WINDIR'), 'explorer.exe')

def explore(path):
    # explorer would choke on forward slashes
    path = os.path.normpath(path)

    if os.path.isdir(path):
        subprocess.run([FILEBROWSER_PATH, path])
    elif os.path.isfile(path):
        subprocess.run([FILEBROWSER_PATH, '/select,', os.path.normpath(path)])


In [18]:
def Appendtext(fname, text):
    with open(fname,'a+') as f:
        f.write(text)
    f.close()

## Directories 

In [3]:
data_dir = join(base_dir, 'data')
tables_dir = join(data_dir, 'tables')
data_long_dir = join(data_dir, 't1_longitudinal')
log_file = join(data_long_dir, 'results', 'failed_pred.txt')
disk_dir = "F:\\"
pred_dir = join(disk_dir, "CoBra\\Data\\volume_longitudinal_nii\\prediction")
with open(join(tables_dir, 'newIDs_dic.pkl'), 'rb') as f:
    id_dic = pickle.load(f)
with open(join(tables_dir, "disk_series_directories.json"), 'rb') as f:
    dir_dic = json.load(f)
inv_id_map = {v: k for k, v in id_dic.items()}
with open(join(data_long_dir, "failed_seg.pkl"), 'rb') as f:
    failed_seg_files = pickle.load(f)
with open(join(data_long_dir, "outliers_affected_br.pkl"), 'rb') as f:
    sids_outliers = pickle.load(f)

df_volume_dir = pd.read_csv(join(tables_dir, 'series_directories.csv'))
sif_volume_dir_dic = pd.Series(
    df_volume_dir.Directory.values, index=df_volume_dir.SeriesInstanceUID)\
        .to_dict()

## Get stroke files

In [4]:
out_ids = [id_dic[sid_stroke] for sid_stroke in sids_outliers]
out_niis_pred = [(join(pred_dir, id+'_1mm.nii.gz'), join(pred_dir, id+'_1mm_seg.nii.gz')) for id in out_ids]

In [5]:
file_ids = [split(vol_file)[1][:6] for vol_file, _ in out_niis_pred]
dcm_dirs = [dir_dic[inv_id_map[file_id]] for file_id in file_ids] 
for dcm_dir in dcm_dirs[:5]:
    DICOMUtils.importDicom(dcm_dir)

In [6]:
print("There are now",len(DICOMUtils.allSeriesUIDsInDatabase()),'files in the database')

There are now 126 files in the database


## Show predictions 

## Nifti

In [ ]:
slicer.mrmlScene.Clear(False)
file_num =75

examine_file_id = None#'008309'
if not isinstance(examine_file_id, type(None)):
    files_vol_seg = [f for f in out_niis_pred if examine_file_id in f[0]]
    print(files_vol_seg)
    vol_file, seg_file = files_vol_seg[0][0], files_vol_seg[0][1]
    print(vol_file)
    file_id = examine_file_id
else:
    vol_file, seg_file = out_niis_pred[file_num]
    file_id = split(vol_file)[1][:6]

volume = slicer.util.loadVolume(vol_file)
seg = slicer.util.loadSegmentation(seg_file)
print(vol_file)
print('new_id =', file_id)
slicernb.ViewDisplay("")
#fig.savefig(join(data_long_dir, 'results','strokes', 'example1.png'))


In [27]:
#exclude_stroke_ids = []
#load exclude stroke ids

In [125]:
exclude_stroke_ids.append(file_id)
print(exclude_stroke_ids)

['001015', '003917', '015017', '019148', '019130', '024408', '025717', '026927', '027622', '032903', '032888', '033936', '035763']


In [130]:
with open(join(data_long_dir, 'results','strokes','exclude_patients_with_large_strokes_ids.pkl'),'wb') as f:
    pickle.dump(exclude_strokes_ids, f)

In [139]:
text = 'ok'
text = '\n'+str(file_id) + ' ' + text
Appendtext(log_file, text)

## Dicom

In [ ]:
dcm_dir = dir_dic[inv_id_map[file_id]]
#dcm_files = [join(dcm_dir, file) for file in os.listdir(dcm_dir)]
dcm_uid = split(dcm_dir)[1]
print(dcm_dir)
print('dcm_uid:', dcm_uid)
print('sif_dir:', sif_volume_dir_dic[dcm_uid])
slicer.mrmlScene.Clear(False)
volume = DICOMUtils.loadSeriesByUID([dcm_uid])
slicernb.ViewDisplay()

In [18]:
print(split(split(vol_file)[0])[0])

F:\CoBra\Data\volume_longitudinal_nii


In [29]:
remove_niis_and_pred = True
if remove_niis_and_pred:
    os.remove(vol_file)
    os.remove(seg_file)
    os.remove(join(split(vol_file)[0], file_id+'_seg.nii.gz'))
    os.remove(join(split(split(vol_file)[0])[0], 'input', 'nii_files', file_id+'.nii.gz'))

## Dynamic 3d 

In [11]:
slicernb.showVolumeRendering(volume, show=True)
app = slicernb.AppWindow()
app

TypeError: GetFirstVolumeRenderingDisplayNode argument 1: method requires a VTK object